# Exercise 33

For this exercise, we will keep track of when our friend’s birthdays are, and be able to find that information based on their name. Create a dictionary (in your file) of names and birthdays. When you run your program it should ask the user to enter a name, and return the birthday of that person back to them.

# Exercise 34

In this exercise, modify your program from Part 1 to load the birthday dictionary from a JSON file on disk, rather than having the dictionary defined in the program.

Bonus: Ask the user for another scientist’s name and birthday to add to the dictionary, and update the JSON file you have on disk with the scientist’s name. If you run the program multiple times and keep adding new names, your JSON file should keep getting bigger and bigger.

# Exercise 35

In this exercise, load that JSON file from disk, extract the months of all the birthdays, and count how many scientists have a birthday in each month.

# Exercise 36

In this exercise, use the bokeh Python library to plot a histogram of which months the scientists have birthdays in! Because it would take a long time for you to input the months of various scientists, you can use my scientist birthday JSON file. Just parse out the months (if you don’t know how, I suggest looking at the previous exercise or its solution) and draw your histogram.

In [68]:
import json
from datetime import datetime
import re
from dateutil import parser
from bokeh.plotting import figure, show

bdayFile = "birthday_file.json"
bDict = {"name":"birthday"}
# {"test":"01/01/2001"}

def openBdayDict(file):
    with open(file,"r") as f:
        bDict = json.load(f)
    return bDict

 
def validateInput(query, validOptions):
    #checks the user's answer to a query against a list of expected input
    #if answer does not match expectations, prompts user to try again
    #valid options can be data types (i.e. str, datetime, etc.) or an RE string
    valid = 0
    toValid = input(query)
    while valid == 0:
        if type(validOptions) == str:
            if re.match(validOptions,toValid.lower()): valid = 1
        elif validOptions == str: valid = 1
        elif validOptions == datetime:
            try:
                toValid = parser.parse(toValid)
                toValid = datetime.strptime(toValid, "%m-%d-%Y")
            except:
                pass
            if type(toValid) == datetime:
                toValid = datetime.strftime(toValid,"%m-%d-%Y")
                valid = 1   
        else: valid = 0
                
        if valid == 0:
            print(toValid)
            toValid = input(f"Whoops! I didn't understand that. Please try again: ")
    return str(toValid)     
    
def lookupBday(file):
    d = openBdayDict(bdayFile)
    addName = validateInput("Whose birthday do you want to look up? ", str)
    try: result = d[addName.lower()]
    except: result = None
    if result != None:
        print(f"{addName}'s birthday is on {d[addName.lower()]}.")
    else:
        print(f"{addName}'s birthday was not found.")
        
def addNewBday(file):
    bDict = openBdayDict(file)
    name = validateInput("What scientist would you like to add a birthday for? ", str)
    lName = name.lower()
    birthday = validateInput("What is or was their birthday? ", datetime)
    newBday = {lName:birthday}
    bDict[lName] = birthday
    with open(file,"w") as f:
        json.dump(bDict,f)
    print(f"OK, {name}'s birthday has been added as {birthday}!")
    
def countBirthdays(file):
    bDict = openBdayDict(file)
    monthCount = {'01':0,'02':0,'03':0,'04':0,'05':0,'06':0,'07':0,'08':0,'09':0,'10':0,'11':0,'12':0}
    for i in bDict.values():
        extractM = "{:02d}".format(int(i[:2]))
        if extractM in monthCount:
            monthCount[extractM] += 1
        else:
            monthCount[i] = '0'
            monthCount[extractM] = 1
    return monthCount

def printBirthdayCount(func, file):
    monthCount = countBirthdays(file)
    print(f"""
    Here is the count of birthdays in each month!
    Jan: {monthCount['01']}
    Feb: {monthCount['02']}
    Mar: {monthCount['03']}
    Apr: {monthCount['04']}
    May: {monthCount['05']}
    Jun: {monthCount['06']}
    Jul: {monthCount['07']}
    Aug: {monthCount['08']}
    Sep: {monthCount['09']}
    Oct: {monthCount['10']}
    Nov: {monthCount['11']}
    Dec: {monthCount['12']}
    """)
    
def makeBirthdayChart (axisDict):
    xList = []    
    for i in axisDict.keys(): #convert our dictionary keys to %b months
        strDate = parser.parse(str(i+'/01/2011'))
        xList.append(datetime.strftime(strDate,"%b"))
    yList = list(axisDict.values())
    p = figure(title='Birthdays of Scientists', 
               x_range = xList,
               x_axis_label = 'Month', 
               y_axis_label = 'Count')
    p.vbar(x=xList, top=yList, width=0.9, bottom=0)
    show(p)

In [317]:
lookupBday(bdayFile)

Whose birthday do you want to look up? Albert Einstein
Albert Einstein's birthday is on 03-14-1879.


In [320]:
addNewBday(bdayFile)

What scientist would you like to add a birthday for? Benjamin Franklin
What is or was their birthday? 01-17-1706
OK, Benjamin Franklin's birthday has been added as 01-17-1706!


In [310]:
printBirthdayCount(countBirthdays, bdayFile)


    Here is the count of birthdays in each month!
    Jan: 1
    Feb: 0
    Mar: 1
    Apr: 0
    May: 0
    Jun: 0
    Jul: 0
    Aug: 0
    Sep: 0
    Oct: 0
    Nov: 0
    Dec: 1
    


In [69]:
makeBirthdayChart(countBirthdays(bdayFile))